## Mutex Lock

In [6]:
import time
from concurrent import futures
import threading
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()
    
    def update(self, name):
        print(f'Thread {name}: starting update')
        with self._lock:
            print(f"Thread {name} has lock")
            local_copy = self.value
            local_copy += 1
            time.sleep(1)
            self.value = local_copy
            print(f"Thread {name} about to release lock")
        print(f'Threading {name}: finishing update')
    

if __name__=="__main__":
    database = FakeDatabase()
    print(f"Testing update. Starting value is {database.value}.")
    with futures.ThreadPoolExecutor(max_workers = 2) as executor:
        for index in range(2):
            executor.submit(database.update, index)
    
    print(f"Testing update. Ending value is {database.value}.")

Testing update. Starting value is 0.
Thread 0: starting updateThread 1: starting update
Thread 0 has lock

Thread 0 about to release lock
Threading 0: finishing update
Thread 1 has lock
Thread 1 about to release lock
Threading 1: finishing update
Testing update. Ending value is 2.


## Semaphore

In [16]:
import threading
from collections import deque
from concurrent import futures
class BoundedBlockingQueue:
    def __init__(self, capacity):
        self.capacity = capacity
        self.queue = deque()
        self.enque_semaphore = threading.Semaphore(capacity)
        self.deque_semaphore = threading.Semaphore(0)
        self._lock = threading.Lock()
    
    def enque(self, val):
        self.enque_semaphore.acquire()
        with self._lock:
            self.queue.append(val)
            print(f'+enqueue:{val}, {self.queue}')
        self.deque_semaphore.release()
    
    def deque(self):
        self.deque_semaphore.acquire()
        with self._lock:
            val = self.queue.popleft()
            print(f'-dequeue:{val}, {self.queue}')
        self.enque_semaphore.release()
        return val
    
    def size(self):
        return len(self.queue)

if __name__=="__main__":
    bbq = BoundedBlockingQueue(5)
    
    with futures.ThreadPoolExecutor(max_workers = 4) as executor:
        for num in range(10):
            executor.submit(bbq.enque, num)
            executor.submit(bbq.deque)
            

+enqueue:0, deque([0])
-dequeue:0, deque([])
+enqueue:1, deque([1])
+enqueue:2, deque([1, 2])
-dequeue:1, deque([2])
+enqueue:3, deque([2, 3])
-dequeue:2, deque([3])
+enqueue:5, deque([3, 5])
-dequeue:3, deque([5])
+enqueue:6, deque([5, 6])
-dequeue:5, deque([6])
+enqueue:7, deque([6, 7])
-dequeue:6, deque([7])
+enqueue:8, deque([7, 8])
-dequeue:7, deque([8])
+enqueue:9, deque([8, 9])
-dequeue:8, deque([9])
-dequeue:9, deque([])
+enqueue:4, deque([4])
-dequeue:4, deque([])


## Coroutine

In [6]:
def bare_bones():
    try:
        while True:
            value = (yield)
            print(value)
    except GeneratorExit:
        print("Exiting coroutine...")

coroutine = bare_bones()
next(coroutine)
coroutine.send('hello world')
coroutine.close()

def filter_line(num):
    while True:
        line = (yield)
        if num in line:
            print(line)

cor = filter_line("33")
next(cor)
cor.send("Jessica, age:24")
cor.send("Marco, age:33")
cor.send("Filipe, age:55")

hello world
Exiting coroutine...
Marco, age:33


## Asyncio

In [12]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

if __name__ == "__main__":
    import time
    s = time.perf_counter()
    await main()
    # asyncio.run(main())
    elapsed = time.perf_counter() - s
    print(f"executed in {elapsed:0.2f} seconds.")

One
One
One
Two
Two
Two
executed in 1.00 seconds.
